In [1]:
"""
    1.建立跟redis的連線
    2.抓secret_key裡面的資料
    3.啟用API伺服器基本樣板，啟用伺服器基本
    4.撰寫用戶關注事件發生時的動作(follow event)
    5.收到按鈕（postback）的封包後(postback event)
    6.針對message event的設定
    7.啟動server

"""

'\n    1.建立跟redis的連線\n    2.抓secret_key裡面的資料\n    3.啟用API伺服器基本樣板，啟用伺服器基本\n    4.撰寫用戶關注事件發生時的動作(follow event)\n    5.收到按鈕（postback）的封包後(postback event)\n    6.針對message event的設定\n    7.啟動server\n\n'

In [2]:
#正式上線時要放在dockerfile中
!pip install line-bot-sdk


In [3]:
import os 

#擷取EC2裡面的ip位置，主要用於test.py
#ip_location=os.environ.get('IPA_ENV')
#請改成自己律定API Server的container name
ip_location='chatbot_api'


In [4]:
"""

    2.抓secret_key裡面的資料（由於是本機執行，所以secret_key可放在本機，之後要上AWS，要放在S3，不能放在github，有安全顧慮）
    

"""
# 載入json處理套件
import json

# 載入基礎設定檔，本機執行所以路徑可以相對位置，在test.py要改成絕對路徑，因為是使用kernel開啟
secretFile=json.load(open("./secret_key",'r'))

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# channel_access_token是用於傳送封包去給line的認證使用類似這個是私鑰，而公鑰已經丟到line那邊，拿著這個就可以有VIP特權
line_bot_api = LineBotApi(secretFile.get("channel_access_token"))

# secret_key是用於當line傳送封包過來時確定line是否為本尊，沒有被仿冒
handler = WebhookHandler(secretFile.get("secret_key"))

# rich_menu_id用於將所指定的rich menu綁定到加好友的用戶上
menu_id = secretFile.get("rich_menu_id")
server_url = secretFile.get("server_url")


In [5]:
"""

  3.啟用伺服器基本樣板，啟用伺服器基本 

"""

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json


# 設定Server啟用細節，這邊使用相對位置，test.py中使用絕對位置，因為是使用kernel開啟
app = Flask(__name__,static_url_path = "/images" , static_folder = "./images/")


# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

#製作一個測試用接口
@app.route('/hello',methods=['GET'])
def hello():
    return 'Hello World!!'

In [6]:
'''

    4.撰寫用戶關注事件發生時的動作
        1. 製作並定義旋轉門選單、flexbubble樣板選單
        2. 取得用戶個資，並存回伺服器
        3. 把先前製作好的自定義菜單，與用戶做綁定
        4. 回應用戶，歡迎用的文字消息、圖片、及旋轉門選單
        5. 製作用戶的redis資料

'''
# 將消息模型，文字收取消息與文字寄發消息，Follow事件引入
from linebot.models import (
    MessageEvent, FollowEvent, JoinEvent,
    TextSendMessage, TemplateSendMessage,
    TextMessage, ButtonsTemplate,
    PostbackTemplateAction, MessageTemplateAction,
    URITemplateAction,ImageSendMessage,CarouselTemplate,CarouselColumn,
    FlexSendMessage,BubbleContainer,StickerSendMessage,ImageSendMessage
)

# 載入requests套件
import requests


#每日熱門推播內容的json格式
flexBubbleContainerJsonString_dailyNews="""
{
    "type": "carousel",
    "contents": [
      {
        "type": "bubble",
        "hero": {
          "type": "image",
          "url": "https://cdn0.tnwcdn.com/wp-content/blogs.dir/1/files/2011/06/BBCNews1.jpg",
          "size": "full",
          "aspectRatio": "20:13",
          "aspectMode": "cover",
          "action": {
            "type": "uri",
            "label": "推播新聞1",
            "uri": "https://www.google.com.tw/"
          }
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "text",
              "text": "熱門新聞",
              "size": "xs",
              "color": "#EA3E0D"
            },
            {
              "type": "text",
              "text": "推播新聞:標題1",
              "size": "xl",
              "weight": "bold",
              "action": {
                "type": "uri",
                "label": "推播新聞1",
                "uri": "https://www.google.com.tw/"
              },
              "wrap": true
            },
            {
              "type": "box",
              "layout": "baseline",
              "flex": 1,
              "contents": [
                {
                  "type": "text",
                  "text": "新聞1:關鍵字",
                  "flex": 0,
                  "size": "lg",
                  "weight": "regular",
                  "action": {
                    "type": "uri",
                    "label": "推播新聞1",
                    "uri": "https://www.google.com.tw/"
                  },
                  "wrap": true
                }
              ]
            },
            {
              "type": "text",
              "text": "新聞來源1",
              "flex": 0,
              "margin": "none",
              "size": "xs",
              "align": "end",
              "gravity": "bottom",
              "color": "#6B69C5",
              "action": {
                "type": "uri",
                "label": "推播新聞3",
                "uri": "https://www.google.com.tw/"
              },
              "wrap": true
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "來源 》新聞標題 1-2",
                "uri": "https://linecorp.com"
              },
              "flex": 2,
              "color": "#AAAAAA",
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "來源 》 新聞標題 1-3",
                "uri": "https://linecorp.com"
              }
            }
          ]
        }
      },
      {
        "type": "bubble",
        "hero": {
          "type": "image",
          "url": "https://cdn0.tnwcdn.com/wp-content/blogs.dir/1/files/2011/06/BBCNews1.jpg",
          "size": "full",
          "aspectRatio": "20:13",
          "aspectMode": "cover",
          "action": {
            "type": "uri",
            "label": "推播新聞2",
            "uri": "https://www.google.com.tw/"
          }
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "text",
              "text": "熱門新聞",
              "size": "xs",
              "color": "#EA3E0D"
            },
            {
              "type": "text",
              "text": "推播新聞:標題2",
              "size": "xl",
              "weight": "bold",
              "action": {
                "type": "uri",
                "label": "推播新聞2",
                "uri": "https://www.google.com.tw/"
              },
              "wrap": true
            },
            {
              "type": "box",
              "layout": "baseline",
              "flex": 1,
              "contents": [
                {
                  "type": "text",
                  "text": "新聞2:關鍵字",
                  "flex": 0,
                  "size": "lg",
                  "weight": "regular",
                  "action": {
                    "type": "uri",
                    "label": "推播新聞2",
                    "uri": "https://www.google.com.tw/"
                  },
                  "wrap": true
                }
              ]
            },
            {
              "type": "text",
              "text": "新聞來源2",
              "flex": 0,
              "margin": "none",
              "size": "xs",
              "align": "end",
              "gravity": "bottom",
              "color": "#6B69C5",
              "action": {
                "type": "uri",
                "label": "推播新聞2",
                "uri": "https://www.google.com.tw/"
              },
              "wrap": true
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "來源 》新聞標題 2-2",
                "uri": "https://linecorp.com"
              },
              "flex": 2,
              "color": "#AAAAAA",
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "來源 》 新聞標題 2-3",
                "uri": "https://linecorp.com"
              }
            }
          ]
        }
      },
      {
        "type": "bubble",
        "hero": {
          "type": "image",
          "url": "https://cdn0.tnwcdn.com/wp-content/blogs.dir/1/files/2011/06/BBCNews1.jpg",
          "size": "full",
          "aspectRatio": "20:13",
          "aspectMode": "cover",
          "action": {
            "type": "uri",
            "label": "推播新聞3",
            "uri": "https://www.google.com.tw/"
          }
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "text",
              "text": "熱門新聞",
              "size": "xs",
              "color": "#EA3E0D"
            },
            {
              "type": "text",
              "text": "推播新聞:標題1",
              "size": "xl",
              "weight": "bold",
              "action": {
                "type": "uri",
                "label": "推播新聞3",
                "uri": "https://www.google.com.tw/"
              },
              "wrap": true
            },
            {
              "type": "box",
              "layout": "baseline",
              "flex": 1,
              "contents": [
                {
                  "type": "text",
                  "text": "新聞2:關鍵字",
                  "flex": 0,
                  "size": "lg",
                  "weight": "regular",
                  "action": {
                    "type": "uri",
                    "label": "推播新聞3",
                    "uri": "https://www.google.com.tw/"
                  },
                  "wrap": true
                }
              ]
            },
            {
              "type": "text",
              "text": "新聞來源3",
              "flex": 0,
              "margin": "none",
              "size": "xs",
              "align": "end",
              "gravity": "bottom",
              "color": "#6B69C5",
              "action": {
                "type": "uri",
                "label": "推播新聞3",
                "uri": "https://www.google.com.tw/"
              },
              "wrap": true
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "來源3-2 》新聞標題3-2",
                "uri": "https://linecorp.com"
              },
              "flex": 2,
              "color": "#AAAAAA",
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "來源3-3 》 新聞標題 3-3",
                "uri": "https://linecorp.com"
              }
            }
          ]
        }
      }
    ]
}"""

In [7]:
flexBubbleContainerJsonString_dailyNews="""
{
    "type": "carousel",
    "contents": [
      {
        "type": "bubble",
        "hero": {
          "type": "image",
          "url": "%s",
          "size": "full",
          "aspectRatio": "20:13",
          "aspectMode": "cover",
          "action": {
            "type": "uri",
            "label": "推播新聞1",
            "uri": "%s"
          }
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "text",
              "text": "熱門新聞",
              "size": "xs",
              "color": "#EA3E0D"
            },
            {
              "type": "text",
              "text": "%s",
              "size": "xl",
              "weight": "bold",
              "action": {
                "type": "uri",
                "label": "推播新聞1",
                "uri": "%s"
              },
              "wrap": true
            },
            {
              "type": "box",
              "layout": "baseline",
              "flex": 1,
              "contents": [
                {
                  "type": "text",
                  "text": "%s",
                  "flex": 0,
                  "size": "lg",
                  "weight": "regular",
                  "wrap": true
                }
              ]
            },
            {
              "type": "text",
              "text": "%s",
              "flex": 0,
              "margin": "none",
              "size": "xs",
              "align": "end",
              "gravity": "bottom",
              "color": "#6B69C5",
              "wrap": true
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "%s",
                "uri": "%s"
              },
              "flex": 2,
              "color": "#AAAAAA",
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "%s",
                "uri": "%s"
              }
            }
          ]
        }
      },
      {
        "type": "bubble",
        "hero": {
          "type": "image",
          "url": "%s",
          "size": "full",
          "aspectRatio": "20:13",
          "aspectMode": "cover",
          "action": {
            "type": "uri",
            "label": "推播新聞2",
            "uri": "%s"
          }
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "text",
              "text": "熱門新聞",
              "size": "xs",
              "color": "#EA3E0D"
            },
            {
              "type": "text",
              "text": "%s",
              "size": "xl",
              "weight": "bold",
              "action": {
                "type": "uri",
                "label": "推播新聞2",
                "uri": "%s"
              },
              "wrap": true
            },
            {
              "type": "box",
              "layout": "baseline",
              "flex": 1,
              "contents": [
                {
                  "type": "text",
                  "text": "%s",
                  "flex": 0,
                  "size": "lg",
                  "weight": "regular",
                  "wrap": true
                }
              ]
            },
            {
              "type": "text",
              "text": "%s",
              "flex": 0,
              "margin": "none",
              "size": "xs",
              "align": "end",
              "gravity": "bottom",
              "color": "#6B69C5",
              "wrap": true
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "%s",
                "uri": "%s"
              },
              "flex": 2,
              "color": "#AAAAAA",
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "%s",
                "uri": "%s"
              }
            }
          ]
        }
      },
      {
        "type": "bubble",
        "hero": {
          "type": "image",
          "url": "%s",
          "size": "full",
          "aspectRatio": "20:13",
          "aspectMode": "cover",
          "action": {
            "type": "uri",
            "label": "推播新聞3",
            "uri": "%s"
          }
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "text",
              "text": "熱門新聞",
              "size": "xs",
              "color": "#EA3E0D"
            },
            {
              "type": "text",
              "text": "%s",
              "size": "xl",
              "weight": "bold",
              "action": {
                "type": "uri",
                "label": "推播新聞3",
                "uri": "%s"
              },
              "wrap": true
            },
            {
              "type": "box",
              "layout": "baseline",
              "flex": 1,
              "contents": [
                {
                  "type": "text",
                  "text": "%s",
                  "flex": 0,
                  "size": "lg",
                  "weight": "regular",
                  "wrap": true
                }
              ]
            },
            {
              "type": "text",
              "text": "%s",
              "flex": 0,
              "margin": "none",
              "size": "xs",
              "align": "end",
              "gravity": "bottom",
              "color": "#6B69C5",
              "wrap": true
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "vertical",
          "spacing": "sm",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "%s",
                "uri": "%s"
              },
              "flex": 2,
              "color": "#AAAAAA",
              "style": "primary"
            },
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "%s",
                "uri": "%s"
              }
            }
          ]
        }
      }
    ]
}"""

In [8]:
#每日熱門推播json格式
flexBubbleContainerJsonString_TLDR="""
{
    "type": "carousel",
    "contents": [
      {
        "type": "bubble",
        "direction": "ltr",
        "header": {
          "type": "box",
          "layout": "vertical",
          "contents": [
            {
              "type": "text",
              "text": "懶人包《 邱議瑩「陪睡說」》",
              "size": "md",
              "align": "center",
              "weight": "bold",
              "color": "#3F3F3F"
            }
          ]
        },
        "hero": {
          "type": "image",
          "url": "https://i.ytimg.com/vi/k8poEUlG_UY/maxresdefault.jpg",
          "size": "full",
          "aspectRatio": "1.51:1",
          "aspectMode": "cover"
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "contents": [
            {
              "type": "text",
              "text": "邱議瑩爆出失言風波，九合一選舉期間，民進黨立委邱議瑩於LINE群組...",
              "size": "xs",
              "align": "start",
              "color": "#5846B6",
              "wrap": true
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "horizontal",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "瞭解更多...",
                "uri": "line://app/1622323314-WBMeekk2"
              },
              "color": "#2EC732",
              "style": "primary"
            }
          ]
        }
      },
      {
        "type": "bubble",
        "direction": "ltr",
        "header": {
          "type": "box",
          "layout": "vertical",
          "contents": [
            {
              "type": "text",
              "text": "懶人包《「 蜂蜜檸檬」吳蕚洋》",
              "size": "md",
              "align": "center",
              "gravity": "center",
              "weight": "bold",
              "color": "#3F3F3F"
            }
          ]
        },
        "hero": {
          "type": "image",
          "url": "https://i.ytimg.com/vi/MGMcgB27y0U/maxresdefault.jpg",
          "size": "full",
          "aspectRatio": "1.51:1",
          "aspectMode": "cover"
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "contents": [
            {
              "type": "text",
              "text": "2018年，台北市長無黨籍候選人吳蕚洋，在電視辯論會上，強力推薦「蜂蜜...",
              "size": "xs",
              "align": "start",
              "weight": "regular",
              "color": "#5846B6",
              "wrap": true
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "horizontal",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "瞭解更多...",
                "uri": "line://app/1622323314-Aag44MMp"
              },
              "color": "#2EC732",
              "style": "primary"
            }
          ]
        }
      },
      {
        "type": "bubble",
        "direction": "ltr",
        "header": {
          "type": "box",
          "layout": "vertical",
          "contents": [
            {
              "type": "text",
              "text": "懶人包《 金馬獎發言風波》",
              "align": "center",
              "weight": "bold",
              "color": "#3F3F3F"
            }
          ]
        },
        "hero": {
          "type": "image",
          "url": "https://static01.hket.com/res/v3/image/content/2210000/2211013/1118001_1024.jpg",
          "size": "full",
          "aspectRatio": "1.51:1",
          "aspectMode": "cover"
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "contents": [
            {
              "type": "text",
              "text": "金馬55版獎典禮上，太陽花紀錄片《我們的青春，在台灣》導演傅榆，上台受獎...",
              "size": "xs",
              "align": "start",
              "color": "#5846B6",
              "wrap": true
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "horizontal",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "瞭解更多..",
                "uri": "line://app/1622323314-XKLeeJJq"
              },
              "color": "#2EC732",
              "style": "primary"
            }
          ]
        }
      },
      {
        "type": "bubble",
        "direction": "ltr",
        "header": {
          "type": "box",
          "layout": "vertical",
          "contents": [
            {
              "type": "text",
              "text": "懶人包《吳敦義「 母豬說」》",
              "size": "md",
              "align": "center",
              "weight": "bold",
              "color": "#3F3F3F"
            }
          ]
        },
        "hero": {
          "type": "image",
          "url": "https://i.ytimg.com/vi/seY1Z9cTiTY/maxresdefault.jpg",
          "size": "full",
          "aspectRatio": "1.51:1",
          "aspectMode": "cover"
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "contents": [
            {
              "type": "text",
              "text": "九合一大選前，吳敦義以國民黨黨主席身分，南下為替高雄市長候選人韓國瑜輔選...",
              "size": "xs",
              "align": "start",
              "weight": "regular",
              "color": "#5846B6",
              "wrap": true
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "horizontal",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "uri",
                "label": "瞭解更多...",
                "uri": "line://app/1622323314-NGZLL77M"
              },
              "color": "#2EC732",
              "style": "primary"
            }
          ]
        }
      }
    ]
}"""

In [9]:
import requests
import json

def get_today_push():
    ip_location='chatbot_api'
    Endpoint='http://%s:5000/push_news' % (ip_location)
    push=requests.get(Endpoint)
    push = push.json()

    def first_push(title_id): 
        Endpoint='http://%s:5000/push_needed/%s'% (ip_location, title_id)
        response=requests.get(Endpoint)
        ans = response.json()
        Endpoint='http://%s:5000/keyword_title/%s'% (ip_location, ans["title"])
        response=requests.get(Endpoint)
        ans['kw'] = response.json()
        ans['kw'] = "、".join(ans['kw'][:3])
        return ans


    def inner_push(title_id): 
        Endpoint='http://%s:5000/push_needed/%s'% (ip_location, title_id)
        response=requests.get(Endpoint)
        ans = response.json()
        if ans['origin'] == 'TVBS新聞台':
            ans['origin'] = 'TVBS'
        elif ans['origin'] == 'SETN三立新聞網':
            ans['origin'] = '三立新聞'

        ans['label'] = ans['origin']+ "》"+ ans['title'][:10] + "..."
        return ans


    a1 = first_push(push['part_a1'])
    a1["url"] = push['part_a1_url']

    a2 = inner_push(push['part_a2'])
    a2["url"] = push['part_a2_url']

    a3 = inner_push(push['part_a3'])
    a3["url"] = push['part_a3_url']

    b1 = first_push(push['part_b1'])
    b1["url"] = push['part_b1_url']

    b2 = inner_push(push['part_b2'])
    b2["url"] = push['part_b2_url']

    b3 = inner_push(push['part_b3'])
    b3["url"] = push['part_b3_url']

    c1 = first_push(push['part_c1'])
    c1["url"] = push['part_c1_url']

    c2 = inner_push(push['part_c2'])
    c2["url"] = push['part_c2_url']

    c3 = inner_push(push['part_c3'])
    c3["url"] = push['part_c3_url']

    flexbu = flexBubbleContainerJsonString_dailyNews % (
        a1["img_url"], a1["url"], a1["title"], a1["url"], a1["kw"], a1["origin"],a2['label'], a2['url'],a3['label'], a3['url'],
        b1["img_url"], b1["url"], b1["title"], b1["url"], b1["kw"], b1["origin"],b2['label'], b2['url'],b3['label'], b3['url'],
        c1["img_url"], c1["url"], c1["title"], c1["url"], c1["kw"], c1["origin"],c2['label'], c2['url'],c3['label'], c3['url'])
    bubbleContainer_dailyNews=CarouselContainer.new_from_json_dict(json.loads(flexbu))
    flexBubbleSendMessage_dailyNews = FlexSendMessage(alt_text="熱門新聞推播",contents=bubbleContainer_dailyNews)
    return flexBubbleSendMessage_dailyNews
    

In [10]:
#將每日新聞推播json格式讀進 FlexSendMessage，並命名flexBubbleSendMessage_dailyNews
#將懶人包json格式讀進FlexSendMessage，並命名flexBubbleSendMessage_TLDR
from linebot.models import(
    FlexSendMessage,BubbleContainer,CarouselContainer,QuickReply,QuickReplyButton,PostbackAction
)

import json
# bubbleContainer_dailyNews=CarouselContainer.new_from_json_dict(json.loads(get_today_push()))
# flexBubbleSendMessage_dailyNews = FlexSendMessage(alt_text="熱門新聞推播",contents=bubbleContainer_dailyNews)

bubbleContainer_TLDR=CarouselContainer.new_from_json_dict(json.loads(flexBubbleContainerJsonString_TLDR))
flexBubbleSendMessage_TLDR = FlexSendMessage(alt_text="懶人包",contents=bubbleContainer_TLDR)

In [11]:
# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):

    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
    
    # 將用戶資訊做成合適Json
    users = {  
        "user_id": user_profile.user_id,
        "display_name": user_profile.display_name,
        "status_message" : user_profile.status_message,
        "picture_url" : user_profile.picture_url
    }
    
    # 將json傳回API Server
    Endpoint='http://%s:5000/users' % (ip_location)
    
    # header要特別註明是json格式
    Header={'Content-Type':'application/json'}
    
    # 傳送post對API server新增資料 
    Response=requests.post(Endpoint,headers=Header,data=json.dumps(users))
    
    #印出Response的資料訊息
    print(Response)
    print(Response.text)


    
    # 將菜單綁定在用戶身上
    # 要到Line官方server進行這像工作，這是官方的指定接口
    linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (user_profile.user_id, menu_id)
    
    # 官方指定的header
    linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFile["channel_access_token"]}
    
    # 傳送post method封包進行綁定菜單到用戶上
    lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)


    #歡迎詞
    #針對剛加入的用戶回覆文字消息、圖片、旋轉門選單
    get_today_push()
    reply_message_list = [
                    TextSendMessage(text="👋我是新聞呆腦獸!😘\n感謝 %s 使用我們的新聞平台❤\n\n🌵🛸每天會推播三則當日最熱門的新聞給您🗿🚀\n\n👉也可以輸入「彩蛋」，查看所有可輸入文字👈\n\n萬一您覺得提醒的次數有點多，您可以在本畫面的聊天室設定選單中，將「提醒」⏰的功能關掉喔！?" % (user_profile.display_name) ),    
                    get_today_push(),
                ] 
    
    #推送訊息給官方Line
    line_bot_api.reply_message(
        event.reply_token,
        reply_message_list    
    )

In [12]:
"""
點選功能選單
懶人包:TLDR   關鍵字查詢:KeyWordSearch 投票結果:VoteResult 區塊鏈查詢:BC_Search
        
"""
from linebot.models import PostbackEvent, URIAction
from urllib.parse import parse_qs


#用戶點擊button後，觸發postback event，對其回傳做相對應處理
@handler.add(PostbackEvent)
def handle_post_message(event):
    #抓取user資料
    user_profile = event.source.user_id
    
    #抓取postback action的data
    data = event.postback.data
    
    data=parse_qs(data)
    print(data)
   
    #點選選單
   
    if data['type'] == ['TLDR']:
        #每要求出題後，redis 的total增加一
        reply_message_list = [
            TextSendMessage(text="⌛正在幫您查詢懶人包...\n\n🎯懶人包的訊息實用嗎❓記得給我們評論哦❗😉 " ),
            flexBubbleSendMessage_TLDR
        ]
        
        line_bot_api.reply_message(event.reply_token, reply_message_list)
        
        
    elif data['type'] == ['KeyWordSearch']:
        
        Endpoint='http://%s:5000/seven_days_kwyword/'% (ip_location)

        response=requests.get(Endpoint)
        kw = response.json()
        quickreply = TextSendMessage(
                        text='選擇關鍵字：',
                        quick_reply=QuickReply(
                            items=[
                                QuickReplyButton(
                                    #使用postback action類似按鈕的概念
                                    action=PostbackAction(label=kw[0],
                                                          #使用了data裝query string的方式，一次裝多個變數
                                                          #這邊使用true_answer()來幫助取得result的值（判斷對或錯）
                                                          data="type=kwd&value=%s" %(kw[0]),
                                                          #按了按鈕之後的會有的回覆
                                                          text=kw[0]
                                                         )
                                ),
                                QuickReplyButton(
                                    action=PostbackAction(label=kw[1],
                                                          data="type=kwd&value=%s" %(kw[1]),
                                                          text=kw[1]
                                                         )
                                ),
                                QuickReplyButton(
                                    action=PostbackAction(label=kw[2],
                                                          data="type=kwd&value=%s" %(kw[2]),
                                                          text=kw[2]
                                                         )
                                ),
                                QuickReplyButton(
                                    action=PostbackAction(label=kw[3],
                                                          data="type=kwd&value=%s" %(kw[3]),
                                                          text=kw[3]
                                                         )
                                ),
                                QuickReplyButton(
                                    action=PostbackAction(label="更多...",
                                                          data="type=more",
                                                          text='更多...'
                                                         )
                                ),
                                
                            ]))
            
        reply_message_list = [
        TextSendMessage(text="👉您可以直接點選下列關鍵字，或按下「更多」輸入您想要查🔍的關鍵字🧐" ),
        quickreply
        ]
        
        line_bot_api.reply_message(event.reply_token, reply_message_list)
        
         
        
    elif data['type'] == ["kwd"]:
        
        keyword_res = data['value'][0] + "連結 :\n"+"https://871e9e6c.ngrok.io/KeyWord3/"+data['value'][0]+"/"
        reply_message_list = [
        TextSendMessage(text=keyword_res)]
        
        line_bot_api.reply_message(event.reply_token, reply_message_list)
        
    elif data['type'] == ['more']:
        reply_message_list = [
        TextSendMessage(text="查更多關鍵字 :\n line://app/1622323314-gVMjjXXG" )]
        
        line_bot_api.reply_message(event.reply_token, reply_message_list)    
    else:
        pass


In [13]:
#彩蛋設計，回覆使用者文字輸入
@handler.add(MessageEvent, message=TextMessage)
#將這次event的參數抓進來
def handle_message(event):
    user_profile = event.source.user_id

    if  event.message.text== "新年快樂":
        line_bot_api.reply_message(
        event.reply_token,
        StickerSendMessage(package_id='1',sticker_id='114'))
        
    elif  ("謝謝" in event.message.text) == True:
        line_bot_api.reply_message(
        event.reply_token,
        StickerSendMessage(package_id='1',sticker_id='2'))
        
    elif event.message.text== "製作團隊":
        line_bot_api.reply_message(
        event.reply_token,
        ImageSendMessage(original_content_url="https://i.imgur.com/v5bpX8b.png",
        preview_image_url="https://i.imgur.com/v5bpX8b.png"))
    
    elif event.message.text== "示範影片":
        reply_message_list = [
        TextSendMessage(text="測試版 :https://www.youtube.com/watch?v=EBRzNqH9etU" )
        ]
        line_bot_api.reply_message(event.reply_token, reply_message_list)
    elif event.message.text== "彩蛋":
        reply_message_list = [
        TextSendMessage(text="💃可輸入「示範影片」、「今日新聞」、「製作團隊」、「彩蛋」、「愛江山更愛美人」、「一隻牛」、「謝謝」、「新年快樂」🕺🏽" )
        ]
        line_bot_api.reply_message(event.reply_token, reply_message_list)
        
#模糊比對，只要輸入中有出現一隻牛就會回覆    
    elif ("一隻牛" in event.message.text) == True:
        reply_message_list = [
            TextSendMessage(text="嗯...讓我思考一下" ),
            TextSendMessage(text="神曲支援 :https://www.youtube.com/watch?v=rkxhQOev7Wo" )
        ]
        
        line_bot_api.reply_message(event.reply_token, reply_message_list)
        
    elif ("愛江山更愛美人" in event.message.text) == True:
        reply_message_list = [
            TextSendMessage(text="嗯...讓我思考一下" ),
            TextSendMessage(text="神曲支援 :https://www.youtube.com/watch?v=91RS6TrnUIc" )
        ]
        
        line_bot_api.reply_message(event.reply_token, reply_message_list)
    elif event.message.text== "今日新聞":
        
        reply_message_list = [
                    TextSendMessage(text="今日上午最熱門的三則新聞" ),    
                    get_today_push()
        ] 
    
        line_bot_api.reply_message(event.reply_token, reply_message_list )
        

In [14]:
import requests
import json

ip_location='chatbot_api'

@app.route("/pushMessage", methods=['GET'])
def push_message():
    Endpoint='http://%s:5000/users_id' % (ip_location)

    Response=requests.get(Endpoint)
    user_id_array = Response.json()
    
    line_bot_api.multicast(
        user_id_array,
        get_today_push()
    )

    return 'OK'

In [15]:
'''
    7.啟動server
    執行此句，啟動Server，觀察後，按左上方塊，停用Server

'''

if __name__ == "__main__":
    app.run(host='0.0.0.0',port=5000)

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.22.0.12 - - [09/Jan/2019 14:47:01] "POST / HTTP/1.1" 200 -


<Response [200]>
{
  "status_describe": "User U29a07f94e16697b2829a39eac85fb46c ('\u9732\u9732 Christina',) is already registered."
}



172.22.0.12 - - [09/Jan/2019 14:47:03] "POST / HTTP/1.1" 200 -
172.22.0.12 - - [09/Jan/2019 14:47:22] "POST / HTTP/1.1" 200 -
172.22.0.12 - - [09/Jan/2019 14:47:28] "POST / HTTP/1.1" 200 -
172.22.0.12 - - [09/Jan/2019 14:50:43] "POST / HTTP/1.1" 200 -
172.22.0.12 - - [09/Jan/2019 14:50:49] "POST / HTTP/1.1" 200 -


<Response [200]>
{
  "status_describe": "success add user"
}



172.22.0.12 - - [09/Jan/2019 15:02:41] "POST / HTTP/1.1" 200 -
172.22.0.12 - - [09/Jan/2019 15:02:55] "POST / HTTP/1.1" 200 -


...

172.22.0.12 - - [11/Jan/2019 20:55:49] "POST / HTTP/1.1" 200 -
172.22.0.12 - - [11/Jan/2019 20:55:49] "POST / HTTP/1.1" 200 -
172.22.0.12 - - [11/Jan/2019 20:56:21] "POST / HTTP/1.1" 200 -
172.22.0.12 - - [11/Jan/2019 20:56:23] "POST / HTTP/1.1" 200 -
172.22.0.12 - - [11/Jan/2019 20:56:33] "POST / HTTP/1.1" 200 -
172.22.0.12 - - [11/Jan/2019 21:06:14] "POST / HTTP/1.1" 200 -


{'type': ['TLDR']}
{'type': ['KeyWordSearch']}


172.22.0.12 - - [11/Jan/2019 21:06:15] "POST / HTTP/1.1" 200 -


{'type': ['KeyWordSearch']}


172.22.0.12 - - [11/Jan/2019 21:06:22] "POST / HTTP/1.1" 200 -
172.22.0.12 - - [11/Jan/2019 21:06:23] "POST / HTTP/1.1" 200 -
172.22.0.12 - - [11/Jan/2019 21:12:32] "POST / HTTP/1.1" 200 -
